# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kiruna,67.8557,20.2251,7.51,100,100,2.06,SE,1690068692
1,1,carloforte,39.1450,8.3058,26.64,80,56,7.38,IT,1690068692
2,2,ribeira grande,38.5167,-28.7000,20.43,84,19,2.68,PT,1690068692
3,3,waitangi,-43.9535,-176.5597,11.57,96,100,2.68,NZ,1690068692
4,4,port elizabeth,-33.9180,25.5701,11.16,82,0,3.09,ZA,1690068692


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City"
)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Drop any rows that have null values
city_data_df = city_data_df.dropna()

# Narrow down cities that fit criteria
city_data_df = city_data_df[(city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] < 27)]
city_data_df = city_data_df[(city_data_df['Wind Speed'] < 4.5)]
city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0)]
city_data_df = city_data_df[(city_data_df['Humidity'] < 50)]

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,boca do acre,-8.7522,-67.3978,26.19,48,0,0.52,BR,1690068700
93,93,gonabad,34.3529,58.6837,22.75,25,0,1.01,IR,1690068701
256,256,sao vicente,-23.9631,-46.3919,21.26,26,0,1.46,BR,1690068521
283,283,manghit,42.1156,60.0597,25.99,29,0,1.48,UZ,1690068716
429,429,chapada dos guimaraes,-15.4606,-55.7497,21.81,34,0,2.44,BR,1690068733
499,499,darwin,-12.4611,130.8418,22.99,38,0,4.12,AU,1690068609
536,536,robore,-18.3333,-59.7500,23.46,34,0,2.27,BO,1690068747


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# # Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# # Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
81,boca do acre,BR,-8.7522,-67.3978,48,
93,gonabad,IR,34.3529,58.6837,25,
256,sao vicente,BR,-23.9631,-46.3919,26,
283,manghit,UZ,42.1156,60.0597,29,
429,chapada dos guimaraes,BR,-15.4606,-55.7497,34,
499,darwin,AU,-12.4611,130.8418,38,
536,robore,BO,-18.3333,-59.7500,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
boca do acre - nearest hotel: No hotel found
gonabad - nearest hotel: هتل پاسارگاد
sao vicente - nearest hotel: Pousada Vitória
manghit - nearest hotel: Mang'it miymanxanasi
chapada dos guimaraes - nearest hotel: Pousada Rios
darwin - nearest hotel: Mantra Pantanas Darwin
robore - nearest hotel: Lajas


,City,Country,Lat,Lng,Humidity,Hotel Name
81,boca do acre,BR,-8.7522,-67.3978,48,No hotel found
93,gonabad,IR,34.3529,58.6837,25,هتل پاسارگاد
256,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
283,manghit,UZ,42.1156,60.0597,29,Mang'it miymanxanasi
429,chapada dos guimaraes,BR,-15.4606,-55.7497,34,Pousada Rios
499,darwin,AU,-12.4611,130.8418,38,Mantra Pantanas Darwin
536,robore,BO,-18.3333,-59.7500,34,Lajas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    hover_cols = ['Country', 'Hotel Name']
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)